# Create Game History for an LLM

If you plan to register LLMs as players, you might want to pass the full game history. This way, the LLM can recognize behavioral patterns of players and make better decisions, perhaps even exploit the weaknesses of the players. In this notebook we define an `EventListener` class that maintains a game history that you can pass to an LLM.

In [1]:
from maverick.players import FoldBot, CallBot, AggressiveBot
from maverick import (
    PlayerLike,
    PlayerState,
    Game,
    GameEvent,
    GameEventType,
    PlayerAction,
    ActionType,
)

In [2]:
class EventListener:
    def __init__(self, game: Game=None):
        self.game = game
        self.events = []
        self.id_to_player = {}
        self.max_line_length = 80

        self.listen(game)
        
    def listen(self, game: Game=None) -> None:
        if game is None:
            return
        
        self.game = game
        self._subscribe_to_events()
        self._register_players()
            
    def _subscribe_to_events(self) -> None:
        for event_type in GameEventType:
            self.game.subscribe(event_type, self._handle_event)

    def _register_players(self) -> None:
        self.id_to_player = {player.id: player for player in self.game.state.players}
    
    @property
    def history(self) -> str:
        return "\n".join(self.events)
    
    def _log_betting_round_started(self) -> None:
        if len(self.game.state.community_cards) == 0:
            self.events.append("Community cards: None")
        else:
            self.events.append(f"Community cards: {[card.code() for card in self.game.state.community_cards]}")
        
        current_bet = self.game.state.current_bet
        current_pot = self.game.state.pot
        last_raise_size = self.game.state.last_raise_size
        self.events.append((
            f"Current pot size: {current_pot}. "
            f"Current bet: {current_bet}. "
            f"Minimum raise size: {last_raise_size}."
        ))
        
        self.events.append("")
    
    def _log_section_header(self, title: str, chr:str, *, add_empty_line:bool=True) -> None:
        len_title = len(title) + 2 if len(title) > 0 else 0
        len_prefix_suffix = (self.max_line_length - len_title) // 2
        title = f" {title} " if len_title > 0 else ""
        msg = chr * len_prefix_suffix + f"{title}" + chr * len_prefix_suffix
        if len(msg) < self.max_line_length:
            msg += chr
        elif len(msg) > self.max_line_length:
            msg = msg[:self.max_line_length]
        msg = msg[:self.max_line_length]
        self.events.append(msg)
        if add_empty_line:
            self.events.append("")
            
    def _handle_event(self, event: GameEvent, game: Game) -> None:
        """Count each type of event."""
        hand_number = game.state.hand_number

        match event.type:
            case GameEventType.GAME_STARTED:
                self._log_section_header("", "=", add_empty_line=False)
                self._log_section_header(f"Game Started", "=", add_empty_line=False)
                self._log_section_header("", "=")
                
                self.events.append("Game type: No-Limit Texas Hold'em")
                self.events.append(f"Small blind: {game.state.small_blind}")
                self.events.append(f"Big blind: {game.state.big_blind}")
                self.events.append(f"Ante: {game.state.ante}")
                self.events.append("")
                
                self.events.append("Players at the table:")
                for player in self.game.state.players:
                    self.events.append(f"    {player.name}: Starting stack {player.state.stack}")
                self.events.append("")
            
            case GameEventType.HAND_STARTED:
                if len(self.events) > 0 and len(self.events[-1].strip()) > 0:
                    self.events.append("")
                
                self._log_section_header(f"Hand {hand_number}", "=")
                
                num_players = len(game.state.players)
                
                button_position = game.state.button_position
                button_player = game.state.players[button_position]
                self.events.append(f"{button_player.name} is on the button.")
                
                small_blind_position = (button_position + 1) % num_players
                small_blind_player = game.state.players[small_blind_position]
                self.events.append(f"{small_blind_player.name} is the small blind.")
                
                big_blind_position = (button_position + 2) % num_players
                big_blind_player = game.state.players[big_blind_position]
                self.events.append(f"{big_blind_player.name} is the big blind.")
                
                utg_position = (button_position + 3) % num_players
                utg_player = game.state.players[utg_position]
                self.events.append(f"{utg_player.name} is under the gun.")
                
                self.events.append("")
                
                self.events.append(f"Player stacks at the beginning of hand {hand_number}:")
                for player in self.game.state.players:
                    self.events.append(f"    {player.name}: {player.state.stack}")
                self.events.append("")
                
            case GameEventType.BETTING_ROUND_STARTED:
                street_name = game.state.street.name
                self._log_section_header(f"Hand: {hand_number} | Street: {street_name}", "-")
                self._log_betting_round_started()

            case GameEventType.PLAYER_ACTION_TAKEN:
                action: PlayerAction = event.action
                player_id = event.player_id
                player = self.id_to_player[player_id]
                current_bet = game.state.current_bet
                current_pot = game.state.pot
                last_raise_size = game.state.last_raise_size

                match action.action_type:
                    case ActionType.FOLD:
                        self.events.append(f"[{player.name}] folds. Remaining stack: {player.state.stack}.")
                    case ActionType.CALL:
                        self.events.append(f"[{player.name}] calls. Remaining stack: {player.state.stack}.")
                    case ActionType.RAISE:
                        self.events.append(f"[{player.name}] raises bet to {current_bet}. Remaining stack: {player.state.stack}.")
                    case ActionType.CHECK:
                        self.events.append(f"[{player.name}] checks. Remaining stack: {player.state.stack}.")
                    case ActionType.BET:
                        self.events.append(f"[{player.name}] bets {action.amount}. Remaining stack: {player.state.stack}.")
                        
                self.events.append((
                    f"Current pot size: {current_pot}. "
                    f"Current bet: {current_bet}. "
                    f"Minimum raise size: {last_raise_size}."
                ))
                        
            case GameEventType.BETTING_ROUND_COMPLETED:
                self.events.append("")
                self.events.append(f"Player standings after the betting round:")
                for player in game.state.players:
                    player_pot = player.state.total_contributed
                    state_type = player.state.state_type.name
                    self.events.append(f"    {player.name}: State={state_type}, Pot={player_pot}, Stack={player.state.stack}")
                self.events.append("")

            case GameEventType.SHOWDOWN_STARTED:
                self._log_section_header(f"Hand: {hand_number} | SHOWDOWN", "-")
                
            case GameEventType.PLAYER_CARDS_REVEALED:
                player_id = event.player_id
                player = self.id_to_player[player_id]
                payload = event.payload
                self.events.append((
                    f"{player.name} reveals: {payload['holding']}. "
                    f"Best hand: {payload['best_hand']} ({payload['best_hand_type']}). "
                    f"Hand score: {payload['best_score']}."
                ))
            
            case GameEventType.SHOWDOWN_COMPLETED:
                self.events.append("")
                self.events.append(f"Player stacks after showdown:")
                for player in game.state.players:
                    self.events.append(f"    {player.name}: {player.state.stack}")
                self.events.append("")
                
            case GameEventType.POT_WON:
                player_id = event.player_id
                player = self.id_to_player[player_id]
                amount = event.payload.get("amount", 0)
                self.events.append(f"{player.name} wins {amount}.")
                
            case GameEventType.PLAYER_ELIMINATED:
                player_id = event.player_id
                player = self.id_to_player[player_id]
                self.events.append(f"{player.name} has been eliminated from the game.")

            case GameEventType.HAND_ENDED:
                pass

In [3]:
game = Game(small_blind=10, big_blind=20, max_hands=10)

players: list[PlayerLike] = [
    CallBot(name="CallBot", state=PlayerState(stack=1000)),
    AggressiveBot(name="AggroBot", state=PlayerState(stack=1000)),
    FoldBot(name="FoldBot", state=PlayerState(stack=1000)),
]

for player in players:
    game.add_player(player)
    

event_listener = EventListener(game)

game.start()

print(event_listener.history)

================================= Game Started =================================

Game type: No-Limit Texas Hold'em
Small blind: 10
Big blind: 20
Ante: 0

Players at the table:
    CallBot: Starting stack 1000
    AggroBot: Starting stack 1000
    FoldBot: Starting stack 1000

==================================== Hand 1 ====================================

CallBot is on the button.
AggroBot is the small blind.
FoldBot is the big blind.
CallBot is under the gun.

Player stacks at the beginning of hand 1:
    CallBot: 1000
    AggroBot: 1000
    FoldBot: 1000

-------------------------- Hand: 1 | Street: PRE_FLOP --------------------------

Community cards: None
Current pot size: 30. Current bet: 20. Minimum raise size: 20.

[CallBot] calls. Remaining stack: 980.
Current pot size: 50. Current bet: 20. Minimum raise size: 20.
[AggroBot] raises bet to 40. Remaining stack: 960.
Current pot size: 80. Current bet: 40. Minimum raise size: 20.
[FoldBot] folds. Remaining stack: 980.
Current pot